In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Conbert Images to Tensors

In [42]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import random
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms #should be used to convert images to pytorch tensors

In [43]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [44]:
trans = transforms.Compose([
    transforms.ToTensor()
])

In [45]:
train = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/CSCE464/datasets/Animals/train_data', transform=trans)

In [46]:
train_loader = torch.utils.data.DataLoader(dataset= train,
                                          batch_size= 10,
                                          shuffle = True,
                                          drop_last = True)

In [47]:
#Create CNN Model
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()


    """
    L1 image in shape = (3, 6, kernel_size=5, stride=1, padding=1) imput channel = 3 for RGB
    Conv -> (100, 64, 14, 14) batch size 100, output channel 64, image size 14x14
    pool -> (100, 64, 7, 7)
    """

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=1),
        #nn.Dropout(0.5),
    )

    """
    L2 image in shape = (100, 32, 14, 14) batch size 100, input channel 32, image size 14x14
    Conv -> (100, 64, 14, 14) batch size 100, output channel 64, image size 14x14
    pool -> (100, 64, 7, 7)
    """

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=1),
        #nn.Dropout(0.5)
    )

    self.fc = torch.nn.Linear(16*58*122, 2, bias=True) #try training model with wrong tensor size to get correct final size
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1) #reduce from 2d data to 1d data
    out = self.fc(out)

    return out

In [48]:
model = CNN().to(device)
test_input = (torch.Tensor(10, 3, 64, 128)).to(device) #batch_size, input channel, image size
test_out = model(test_input)

In [49]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss().to(device)

In [50]:
total_batch = len(train_loader)
print(total_batch)

50


In [51]:
epochs = 20
for epoch in range(epochs):
  avg_cost = 0
  for idx, data in enumerate(train_loader):
    x_train, y_train = data
    x_train = x_train.to(device)
    y_train = y_train.to(device)

    optimizer.zero_grad()
    pred = model(x_train)
    cost = loss(pred, y_train)

    correct_pred = torch.argmax(pred, 1) == y_train
    accuracy = correct_pred.float().mean()

    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("[Epoch: {}] Cost = {} Training ACC: {:.2f}".format(epoch+1, avg_cost, accuracy*100))


[Epoch: 1] Cost = 9.221903800964355 Training ACC: 60.00
[Epoch: 2] Cost = 1.16202712059021 Training ACC: 40.00
[Epoch: 3] Cost = 1.0007407665252686 Training ACC: 40.00
[Epoch: 4] Cost = 0.9170339107513428 Training ACC: 70.00
[Epoch: 5] Cost = 0.8297920823097229 Training ACC: 40.00
[Epoch: 6] Cost = 0.8064319491386414 Training ACC: 50.00
[Epoch: 7] Cost = 0.7344295382499695 Training ACC: 80.00
[Epoch: 8] Cost = 0.7215409278869629 Training ACC: 40.00
[Epoch: 9] Cost = 0.6954723000526428 Training ACC: 30.00
[Epoch: 10] Cost = 0.6956565976142883 Training ACC: 70.00
[Epoch: 11] Cost = 0.6944122910499573 Training ACC: 60.00
[Epoch: 12] Cost = 0.696286678314209 Training ACC: 20.00
[Epoch: 13] Cost = 0.6941996812820435 Training ACC: 60.00
[Epoch: 14] Cost = 0.6937079429626465 Training ACC: 60.00
[Epoch: 15] Cost = 0.6940815448760986 Training ACC: 30.00
[Epoch: 16] Cost = 0.6938849091529846 Training ACC: 60.00
[Epoch: 17] Cost = 0.694331169128418 Training ACC: 50.00
[Epoch: 18] Cost = 0.6955281

In [53]:
torch.save(model.state_dict(), '/content/drive/MyDrive/CSCE464/ModelSaving/CNN_Animals.pth') #end is filename.pth

In [54]:
#load model instead of having to train again
new_model = CNN().to(device)
new_model.load_state_dict(torch.load('/content/drive/MyDrive/CSCE464/ModelSaving/CNN_Animals.pth'))

<All keys matched successfully>

In [55]:
test = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/CSCE464/datasets/Animals/test_data', transform=trans)

In [56]:
test_loader = torch.utils.data.DataLoader(dataset= test,
                                          batch_size= len(test), #must always be full batch size
                                          shuffle = True,
                                          drop_last = True)

#Testing model

In [59]:
new_model.eval()

with torch.no_grad():
  for idx, data in enumerate(test_loader):
    x_test, y_test = data
    x_test = x_test.to(device)
    y_test = y_test.to(device)

  pred = new_model(x_test)
  correct_prediction = torch.torch.argmax(pred, axis=1) == y_test
  accuracy = correct_prediction.float().mean()
  print("Testing Accuracy {:.2f}".format(accuracy.item()*100))

Testing Accuracy 18.38
